### 垃圾短信分类
语料是短信消息，分为两类，分别是正常的短信，如祝福消息、验证码消息等一些有用消息，标签为0；另一类是垃圾消息，如广告、推销等垃圾信息，标签为1。且标签与文本内容之间使用'\t'分开

In [1]:
import jieba
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer      #使用Tfidf进行文本的向量化处理，提取文本特征

In [2]:
def read_data(data_path):
    """
    读取数据
    :param data_path: 数据存放路径
    :return: 读取到的数据
    """
    with open(data_path, 'r', encoding='utf-8') as f:
        data = f.readlines()
    return data

In [3]:
def cut_words(data,stopwords, test_size=0.3):
    """
    分词、去停用词并将数据集分成训练集和测试集
    :param data: 文本数据
    :param stopwords: 停用词
    :param test_size: 测试集的比例
    :return: 测试集和训练集
    """
    stop_words = list()
    for word in stopwords:
        stop_words.append(word[:-1])
    y = list()
    text_list = list()
    for line in data:
        label, text = line.split('\t', 1)     #num=1，表示分割1次，返回两个列表，分别为label和text
        cut_text = [word for word in jieba.cut(text) if word not in stop_words]
        if cut_text == '':
            continue
        else:
            text_list.append(' '.join(cut_text))
            y.append(int(label))
    return sklearn.model_selection.train_test_split(text_list, y, test_size=test_size, random_state=10)

In [4]:
def calculate_tfidf(X_train, X_test):
    """
    计算文本的tf-idf
    :param X_train: 训练集
    :param X_test: 测试集
    :return: 返回的是文本的tf-idf特征向量
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(X_train)
    X_train_tfidf = vectorizer.transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    return X_train_tfidf, X_test_tfidf, vectorizer

In [5]:
def evaluate(model, X, y):
    """
    模型评估
    :param model: 训练好的模型
    :param X: 测试集
    :param y: 测试集标签
    :return: 正确率和auc值
    """
    accuracy = model.score(X, y)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y, model.predict_proba(X)[:, 1], pos_label=1)
    return accuracy, sklearn.metrics.auc(fpr, tpr)

### 以下为文本分类建模

In [6]:
# step1 读取数据（文本和停用词）
data_path = "./data/train.txt"
stopwords_path = "./data/stopwords.txt"
data = read_data(data_path)
stopwords = read_data(stopwords_path)

In [7]:
# step2 分词、分为训练集和测试集
X_train, X_test, y_train, y_test = cut_words(data, stopwords, test_size=0.3)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MSWQPC\AppData\Local\Temp\jieba.cache
Loading model cost 1.395 seconds.
Prefix dict has been built successfully.


In [8]:
# step3 提取特征参数（tf-idf）
X_train_tfidf, X_test_tfidf, tfidf_model = calculate_tfidf(X_train, X_test)

In [9]:
# step4 训练lr模型
lr = LogisticRegression(C=0.5)
lr.fit(X_train_tfidf, y_train)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# step5 模型评估
accuracy, auc = evaluate(lr, X_train_tfidf, y_train)
print("训练集正确率：%.4f%%\n" % (accuracy * 100))
print("训练集AUC值：%.6f\n" % auc)

accuracy, auc = evaluate(lr, X_test_tfidf, y_test)
print("测试集正确率：%.4f%%\n" % (accuracy * 100))
print("测试AUC值：%.6f\n" % auc)

训练集正确率：92.7223%

训练集AUC值：0.993224

测试集正确率：90.9174%

测试AUC值：0.980725

